In [9]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [10]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [11]:
portfolio = portfolio.explode('channels')
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,email,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
0,10,mobile,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
0,10,social,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,web,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
1,10,email,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
1,10,mobile,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
1,10,social,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,web,0,4,informational,3f207df678b143eea3cee63160fa8bed
2,0,email,0,4,informational,3f207df678b143eea3cee63160fa8bed
2,0,mobile,0,4,informational,3f207df678b143eea3cee63160fa8bed


In [13]:
profile

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
16997,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [15]:
profile.groupby( ['gender'], dropna=False ).count()

,age,id,became_member_on,income
gender,,,,
F,6129,6129,6129,6129
M,8484,8484,8484,8484
O,212,212,212,212
NaN,2175,2175,2175,0


In [16]:
transcript

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [12]:
# conn = create_engine('sqlite:///data/raw_starbucks.db')

# portfolio.to_sql('portfolio', conn, if_exists='replace', index=False)
# profile.to_sql('profile', conn, if_exists='replace', index=False)
# transcript.to_sql('transcript', conn, if_exists='replace', index=False)

# conn.dispose()